In [1]:
!pip install transformers[torch]
!pip install accelerate -U
!pip install datasets[audio]
!pip install evaluate
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 14.9 MB/s eta 0:00:00


In [2]:
import numpy as np
import IPython.display as ipd
import torch
import transformers
import datasets
import librosa
import re
import json
import random
import matplotlib.pyplot as plt
import pandas as pd
import evaluate
import nltk

from IPython.display import display, HTML
from datasets import load_dataset, load_metric, ClassLabel, Audio
from transformers import WhisperConfig, WhisperProcessor, WhisperFeatureExtractor, WhisperTokenizer, WhisperForConditionalGeneration
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding, DataCollator
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, Trainer, TrainingArguments
from transformers import pipeline
from transformers.utils import send_example_telemetry

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from transformers import AdamW

# from rouge import Rouge

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Load Datasets

In [3]:
whisper = "openai/whisper-tiny"
minds14 = "PolyAi/minds14"

In [4]:
datasets = load_dataset(
    minds14,
    name="en-AU",
    split="train"
    )

# !huggingface-cli download PolyAI/minds14 --repo-type dataset --revision refs/convert/parquet --local-dir . --local-dir-use-symlinks False  --include 'en-US/*'
# from datasets import load_dataset
# datasets = load_dataset('./en-US', split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
datasets

Dataset({
    features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
    num_rows: 654
})

# Data Preprocess

In [6]:
datasets = datasets.train_test_split(test_size = 0.2)
print("split: ", datasets)

datasets = datasets.remove_columns(
    ["english_transcription",
    "intent_class",
    "lang_id",]
)
print("remove column: ", datasets)

print(datasets['train']['path'][0])
print(datasets['train']['audio'][0]['array'])
print(datasets['train']['audio'][0]['sampling_rate'])
print(datasets['train']['transcription'][0])


split:  DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 523
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 131
    })
})
remove column:  DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription'],
        num_rows: 523
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription'],
        num_rows: 131
    })
})
/root/.cache/huggingface/datasets/downloads/extracted/a19fbc5032eacf25eab0097832db7b7f022b42104fbad6bd5765527704a428b9/en-AU~CARD_ISSUES/CA79ed67077daa44579a5049cc3b2506f9_3.wav
[-0.00024414 -0.00024414 -0.00024414 ...  0.          0.
  0.        ]
8000
also I recently growcott on you but I'm doesn't work when I tried tattoo all-time use it to pay it was declined and I was wondering if I could get some help thank you


## Random Check datasets Transcription

In [7]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))


show_random_elements(dataset=datasets['train'].remove_columns(['audio', 'path']), num_examples=10)

,transcription
0,how can I set up a joint account
1,cash can I withdraw at an ATM
2,hi I'm just calling you at a question with an existing business loan that I have
3,could I please see my current account balance
4,a s good morning I just want to put a block on my ATM card at the moment please
5,show me my direct debit
6,my car doesn't seem to work
7,I was wondering how I can view my account balance
8,I'm just wondering what's the maximum amount that I can withdraw from the ATM
9,please show me my account balance


# remove punctuation

In [8]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_ignore_regex, '', batch["transcription"]).lower() + " "
    batch["transcription"] = batch['transcription'].rstrip()
    return batch

datasets = datasets.map(remove_special_characters)

Map:   0%|          | 0/523 [00:00<?, ? examples/s]

Map:   0%|          | 0/131 [00:00<?, ? examples/s]

In [9]:
show_random_elements(dataset=datasets['train'].remove_columns(['audio', 'path']), num_examples=10)

,transcription
0,i would like to deposit money into my account
1,i need to know what's like my bank i can work overseas
2,hi i'm trying to make a large transaction online with my bank account and it says i'm getting a text message for verification
3,hello i was hoping to get some information about what i need to do to apply for a business loan
4,i need my card to work abroad
5,the app for my bank is not working at the moment how can i access my money
6,clip to stop all the transaction on my card i just lost it so i need to freeze my card and put a block on any transactions over the last 24 hours as i lost my card and it needs to be this big locked
7,hi i'm traffic lights payment prefer lots of money i do not know what to do with my messages and apparently chords
8,yes i'm i wouldn't know if i can use my credit card when i'm overseas please
9,hi there i'm not able to use my card it's not working at the moment why was my payment to kind


In [10]:
def extract_all_chars(batch):
  all_text = " ".join(batch["transcription"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [11]:
vocabs = datasets.map(
    extract_all_chars,
    batched=True,
    batch_size=1,
    keep_in_memory=True,
    remove_columns=datasets.column_names['train']
)

vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
print(vocab_dict)

vocab_dict["|"] = vocab_dict[" "]
del vocab_dict [" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

Map:   0%|          | 0/523 [00:00<?, ? examples/s]

Map:   0%|          | 0/131 [00:00<?, ? examples/s]

{'r': 0, 'k': 1, 'o': 2, 'b': 3, 't': 4, ' ': 5, 'p': 6, 's': 7, 'e': 8, 'u': 9, 'l': 10, 'a': 11, 'w': 12, 'm': 13, 'i': 14, 'z': 15, 'c': 16, 'y': 17, 'n': 18, 'h': 19, 'f': 20, "'": 21, 'd': 22, 'g': 23}
26


In [12]:
# with open('vocab.json', 'w') as vocab_file:
#     json.dump(vocab_dict, vocab_file)

## change sampling_rate audio

In [13]:
rand_int = random.randint(0, len(datasets['train']))

print(datasets['train'][rand_int]['transcription'])
ipd.Audio(data=np.asarray(datasets['train'][rand_int]['audio']['array']), autoplay=True, rate=8000)

i've just received my credit card bill and i don't understand how you pay it can you help me


In [14]:
rand_int = random.randint(0, len(datasets['train']))

print(datasets['train'][rand_int]['transcription'])
ipd.Audio(data=np.asarray(datasets['train'][rand_int]['audio']['array']), autoplay=True, rate=8000)

g'day i'm not going to set up a joint account with my partner can you help me


In [15]:
datasets = datasets.cast_column("audio", Audio(sampling_rate=16_000))
datasets['train'][0]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/a19fbc5032eacf25eab0097832db7b7f022b42104fbad6bd5765527704a428b9/en-AU~CARD_ISSUES/CA79ed67077daa44579a5049cc3b2506f9_3.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/a19fbc5032eacf25eab0097832db7b7f022b42104fbad6bd5765527704a428b9/en-AU~CARD_ISSUES/CA79ed67077daa44579a5049cc3b2506f9_3.wav',
  'array': array([-2.14414235e-04, -2.71049299e-04, -2.75653554e-04, ...,
         -2.13041185e-05, -1.74164034e-05,  1.57961385e-05]),
  'sampling_rate': 16000},
 'transcription': "also i recently growcott on you but i'm doesn't work when i tried tattoo alltime use it to pay it was declined and i was wondering if i could get some help thank you"}

# Extract datasets

In [16]:
tokenizer = WhisperTokenizer.from_pretrained(whisper, language="english")
feature_extractor = WhisperFeatureExtractor.from_pretrained(whisper)
processor = WhisperProcessor(
    feature_extractor=feature_extractor,
    tokenizer=tokenizer,
    )

model = WhisperForConditionalGeneration.from_pretrained(whisper)
model.config.forced_decoder_ids = None

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.72k [00:00<?, ?B/s]

### test processor

In [17]:
audio_array = datasets['train']['audio'][0]['array']
audio_sr = datasets['train']['audio'][0]['sampling_rate']
audio_text = datasets['train']['transcription'][0]

encoded_input = processor(
    audio_array,
    sampling_rate=audio_sr,
    return_tensors='pt',
).input_features

encoded_label = processor(
    text=audio_text,
    return_tensors='pt',
).input_ids

print("array: ", audio_array)
print("text: ", audio_text)
print("#"*50)
print("feature: ", encoded_input)
print("label: ", encoded_label)

array:  [-2.14414235e-04 -2.71049299e-04 -2.75653554e-04 ... -2.13041185e-05
 -1.74164034e-05  1.57961385e-05]
text:  also i recently growcott on you but i'm doesn't work when i tried tattoo alltime use it to pay it was declined and i was wondering if i could get some help thank you
##################################################
feature:  tensor([[[-0.6290, -0.5185, -0.3286,  ..., -0.7714, -0.7714, -0.7714],
         [-0.6604, -0.7714, -0.3188,  ..., -0.7714, -0.7714, -0.7714],
         [-0.4593, -0.5144, -0.4076,  ..., -0.7714, -0.7714, -0.7714],
         ...,
         [-0.7714, -0.7714, -0.7714,  ..., -0.7714, -0.7714, -0.7714],
         [-0.7714, -0.7714, -0.7714,  ..., -0.7714, -0.7714, -0.7714],
         [-0.7714, -0.7714, -0.7714,  ..., -0.7714, -0.7714, -0.7714]]])
label:  tensor([[50258, 50259, 50363, 41551,   741,  3938,  1852, 34000,   322,   291,
           457,   741,   478,  1177,   380,   589,   562,   741,   220,    83,
          2428,   220, 27321, 32599,   439,  37

### model generate test

In [18]:
predicted_ids = model.generate(encoded_input)

### decode test processor

In [19]:
decoded_input = processor.batch_decode(predicted_ids, skip_special_tokens=True)
decoded_label = processor.batch_decode(encoded_label, skip_special_tokens=True)

print("decoded input: ", decoded_input)
print("decoded label: ", decoded_label)

decoded input:  [" So I recently got a card from you, but it doesn't work when I took the top of it or I'm used to pay it, or it gets declined, and I was wondering if I could get them help. Thank you."]
decoded label:  ["also i recently growcott on you but i'm doesn't work when i tried tattoo alltime use it to pay it was declined and i was wondering if i could get some help thank you"]


# Extract Datasets

In [20]:
def prepare_datasets(batch):
  audio = batch['audio']

  batch = processor(
      audio=audio['array'],
      sampling_rate=audio['sampling_rate'],
      text=batch['transcription'],
  )

  batch['input_length'] = len(audio['array']) / audio['sampling_rate']

  return batch

In [21]:
encoded_datasets = datasets.map(
    prepare_datasets,
    remove_columns=datasets.column_names['train'],
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/523 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/131 [00:00<?, ? examples/s]

In [22]:
print(encoded_datasets)

print(np.shape(encoded_datasets['train']['input_features']))
print(np.shape(encoded_datasets['train']['labels']))

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels', 'input_length'],
        num_rows: 523
    })
    test: Dataset({
        features: ['input_features', 'labels', 'input_length'],
        num_rows: 131
    })
})
(523, 1, 80, 3000)
(523,)


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:2009: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


# Prepare model for training

### Data Collator

In [23]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [
            {"input_features": feature["input_features"][0]} for feature in features
        ]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [24]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

### Metrics

In [25]:
metric_wer = evaluate.load("wer")

In [26]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    wer_ortho = 100 * metric_wer.compute(predictions=pred_str, references=label_str)

    pred_str_norm = [normalizer(pred) for pred in pred_str]
    label_str_norm = [normalizer(label) for label in label_str]
    pred_str_norm = [
        pred_str_norm[i] for i in range(len(pred_str_norm)) if len(label_str_norm[i]) > 0
    ]
    label_str_norm = [
        label_str_norm[i]
        for i in range(len(label_str_norm))
        if len(label_str_norm[i]) > 0
    ]

    wer = metric_wer.compute(predictions=pred_str_norm, references=label_str_norm)

    return {"wer_ortho": wer_ortho, "wer": wer}

In [27]:
from functools import partial

model.config.use_cache = False
model.generate = partial(
    model.generate,
    language="english",
    task="transcribe",
    use_cache=True,
)

optimizer = AdamW(
    model.parameters(),
    lr=1e-3,
    eps=1e-8,
    betas=(0.9, 0.999),
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# Training

In [28]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-tiny-minds14",
    num_train_epochs=4,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=50,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    fp16_full_eval=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=500,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    # push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    args= training_args,
    model=model,
    train_dataset=encoded_datasets['train'],
    eval_dataset=encoded_datasets['test'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,
    # optimizers=optimizer, # default optimizer is AdamW
)

In [29]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer Ortho,Wer
500,0.567600,0.475894,19.321149,0.193176
1000,0.010300,0.538078,18.798956,0.188159
1500,0.000700,0.568970,18.485640,0.185650
2000,0.000200,0.587885,18.381201,0.184646
2500,0.000100,0.602116,18.433420,0.184646
3000,0.000100,0.613757,18.642298,0.186152
3500,0.000100,0.628073,18.694517,0.186653
4000,0.000100,0.638876,19.112272,0.190667


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=4000, training_loss=0.07240901727834717, metrics={'train_runtime': 2974.0269, 'train_samples_per_second': 5.38, 'train_steps_per_second': 1.345, 'total_flos': 3.931635953664e+17, 'train_loss': 0.07240901727834717, 'epoch': 30.53})

In [30]:
# trainer.evaluate()

In [31]:
# trainer.predict(test_dataset=encoded_datasets['test'])

In [38]:
saved_model_pt = "whisper_asr_minds14_enAU"

trainer.save_model(saved_model_pt)

# Predict and Evaluate

In [39]:
# audio_file = datasets['test']['path'][0]
device = "cuda:0" if torch.cuda.is_available() else "cpu"

transcriber = pipeline(
    "automatic-speech-recognition",
    model=saved_model_pt,
    chunk_length_s=30,
    device=device,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [40]:
# data_test = load_dataset("google/fleurs", name='en_us', split='test')
data_test = load_dataset("PolyAi/minds14", name="en-AU", split="train")

data_test

Dataset({
    features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
    num_rows: 654
})

In [41]:
data_test

Dataset({
    features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
    num_rows: 654
})

In [42]:
audio_file = data_test['path'][0]
audio_file
# audio_file = "/root/.cache/huggingface/datasets/downloads/extracted/767809e2794806f54865259a100b1774ae3de84135396cf5c5fa2a50bd3bcbef/test/1003119935936341070.wav"

'/root/.cache/huggingface/datasets/downloads/extracted/a19fbc5032eacf25eab0097832db7b7f022b42104fbad6bd5765527704a428b9/en-AU~PAY_BILL/response_4.wav'

In [43]:
prediction = transcriber(audio_file, batch_size=8, return_timestamps=True)['chunks']

print(prediction)

[{'timestamp': (0.0, 9.2), 'text': 'i would like to pay my electricity bill using my card can you please assist'}]


In [46]:
print(data_test['transcription'][0])
ipd.Audio(data=np.asarray(data_test['audio'][0]['array']), rate=16000)

I would like to pay my electricity bill using my card can you please assist


In [47]:
prediction[0]['text']

'i would like to pay my electricity bill using my card can you please assist'

In [48]:
pred_words = prediction[0]['text'].split()
ref_words = data_test['transcription'][0].split()

wer = metric_wer.compute(predictions=pred_words, references=ref_words)

wer_presentage = wer * 100
print(f"WER: {wer_presentage:.2f}%")

WER: 6.67%


In [49]:
# trainer.save_model("/content/drive/MyDrive/whisper-asr-minds14-enUS")

In [50]:
# from datasets import Dataset

# data_test = load_dataset("PolyAi/minds14", name="en-AU", split="train")

# data_test

In [51]:
data_test = data_test.train_test_split(test_size=0.2)
data_test

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 523
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 131
    })
})

In [52]:
audio_file = data_test['test']['path'][0]
target = data_test['test']['transcription'][0]

audio_file
prediction = transcriber(
    audio_file,
    batch_size=8,
    return_timestamps=True,
)['chunks']

print("timestamp: ", prediction[0]['timestamp'])
print("predict: ", prediction[0]['text'])
print("target: ", target)

timestamp:  (0.0, 12.8)
predict:   good afternoon i'm ring up wishing to change my address i've actually just recently moved and i would like to just buy you
target:  good afternoon I'm ringing up wishing to change my address I've actually just recently moved and I would like to just voice you this so you can update and therefore you are where where I am now


In [53]:
# rouge_metric = load_metric('rouge')

In [54]:
def predict_eval(audio_file, target, idx):
    preds = transcriber(
        audio_file,
        batch_size=8,
    )['text']

    print("preds: ", preds)
    print("target: ", target)

    pred_words = preds.split(" ")
    ref_words = target.split(" ")

    # Remove any empty strings from the end of the lists
    if pred_words[-1] == '':
        pred_words = pred_words[:-1]
    if ref_words[-1] == '':
        ref_words = ref_words[:-1]

    # Pad the shorter sequence with empty strings
    max_length = max(len(pred_words), len(ref_words))
    pred_words += ['PAD'] * (max_length - len(pred_words))
    ref_words += ['PAD'] * (max_length - len(ref_words))

    print(len(pred_words))
    print(len(ref_words))
    print(pred_words)
    print(ref_words)

    # Check if both prediction and reference are non-empty
    if pred_words and ref_words:
        wer = metric_wer.compute(predictions=pred_words, references=ref_words)
        wer_percentage = wer * 100
    else:
        wer_percentage = 0.0  # or any other default value

    # Compute accuracy
    correct = sum(1 for p, r in zip(pred_words, ref_words) if p == r)
    accuracy = (correct / len(pred_words)) * 100

    return {
        "num": idx,
        "wer": f"{wer_percentage:.2f}%",
        "accuracy": accuracy,
        "predict": preds,
        "target": target,
    }



preds_eval_list = []

for idx in range(len(data_test['test'])):

  predicted = predict_eval(data_test['test']['path'][idx], data_test['test']['transcription'][idx], idx)
  preds_eval_list.append(predicted)

# print(preds_eval_list)

preds:  good afternoon i'm ring up wishing to change my address i've actually just recently moved and i would like to just buy you this is their you can update and there for you are aware where i am now thank you very much
target:  good afternoon I'm ringing up wishing to change my address I've actually just recently moved and I would like to just voice you this so you can update and therefore you are where where I am now
43
43
['good', 'afternoon', "i'm", 'ring', 'up', 'wishing', 'to', 'change', 'my', 'address', "i've", 'actually', 'just', 'recently', 'moved', 'and', 'i', 'would', 'like', 'to', 'just', 'buy', 'you', 'this', 'is', 'their', 'you', 'can', 'update', 'and', 'there', 'for', 'you', 'are', 'aware', 'where', 'i', 'am', 'now', 'thank', 'you', 'very', 'much']
['good', 'afternoon', "I'm", 'ringing', 'up', 'wishing', 'to', 'change', 'my', 'address', "I've", 'actually', 'just', 'recently', 'moved', 'and', 'I', 'would', 'like', 'to', 'just', 'voice', 'you', 'this', 'so', 'you', 'can

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


preds:  stop all transactions on my card
target:  stop all transactions on my card
6
6
['stop', 'all', 'transactions', 'on', 'my', 'card']
['stop', 'all', 'transactions', 'on', 'my', 'card']


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


preds:  i need to clarify this most recent transaction
target:  I need to clarify this most recent transaction
8
8
['i', 'need', 'to', 'clarify', 'this', 'most', 'recent', 'transaction']
['I', 'need', 'to', 'clarify', 'this', 'most', 'recent', 'transaction']


...

preds:  hi there i'm just calling in regards to the limit at the atm is there a way i can increase my withdrawal limit at the cash machine
target:  hi there I'm just calling in regards to the limit at the ATM is there a way I can increase my withdrawal limit at the cash machine
27
27
['hi', 'there', "i'm", 'just', 'calling', 'in', 'regards', 'to', 'the', 'limit', 'at', 'the', 'atm', 'is', 'there', 'a', 'way', 'i', 'can', 'increase', 'my', 'withdrawal', 'limit', 'at', 'the', 'cash', 'machine']
['hi', 'there', "I'm", 'just', 'calling', 'in', 'regards', 'to', 'the', 'limit', 'at', 'the', 'ATM', 'is', 'there', 'a', 'way', 'I', 'can', 'increase', 'my', 'withdrawal', 'limit', 'at', 'the', 'cash', 'machine']


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


preds:  hi i'm looking to get my account balance
target:  hi I'm looking to get my account balance
8
8
['hi', "i'm", 'looking', 'to', 'get', 'my', 'account', 'balance']
['hi', "I'm", 'looking', 'to', 'get', 'my', 'account', 'balance']


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


preds:  hi how would i go about changing my address
target:  hi how would I go about changing my address
9
9
['hi', 'how', 'would', 'i', 'go', 'about', 'changing', 'my', 'address']
['hi', 'how', 'would', 'I', 'go', 'about', 'changing', 'my', 'address']


In [55]:
preds_eval_list

[{'num': 0,
  'wer': '55.81%',
  'accuracy': 44.18604651162791,
  'predict': "good afternoon i'm ring up wishing to change my address i've actually just recently moved and i would like to just buy you this is their you can update and there for you are aware where i am now thank you very much",
  'target': "good afternoon I'm ringing up wishing to change my address I've actually just recently moved and I would like to just voice you this so you can update and therefore you are where where I am now"},
 {'num': 1,
  'wer': '18.18%',
  'accuracy': 81.81818181818183,
  'predict': "the app isn't working even when i refresh it not loading",
  'target': "the App isn't working even when I refresh it not loading"},
 {'num': 2,
  'wer': '7.69%',
  'accuracy': 92.3076923076923,
  'predict': 'hi just calling to ask how i can transfer money into the account',
  'target': 'hi just calling to ask how I can transfer money into the account'},
 {'num': 3,
  'wer': '0.00%',
  'accuracy': 100.0,
  'predict

In [56]:
accuracy_values = [entry['accuracy'] for entry in preds_eval_list]
overall_accuracy = sum(accuracy_values) / len(accuracy_values)

print(f"Overall Accuracy: {overall_accuracy:.2f}%")

Overall Accuracy: 83.72%


In [ ]:
# target_ = data_test['test']['transcription'][0].split(" ")
# print(target_)
# print(len(target_))

In [57]:
pred, lbl, _ = trainer.predict(test_dataset=encoded_datasets['test'])

RuntimeError: ignored

In [58]:
text = "can you tell me what my current account balance is please "
text_without_spaces_at_end = text.rstrip()

print(text_without_spaces_at_end)

can you tell me what my current account balance is please
